# Команда "Крылья" - проект "Лёгкий старт"
##### Участники: Артур Сапрыкин, Иван Рычков
- Мы искали такой подход к решению задачи, логику которого можно легко объяснить, и у нас самих не возникает путаницы в происходящем.
- Постарались извлечь максимум пользы из тех данных, которые были нам предложены, и нам удалось найти в них взаимосвязи.

In [1]:
# Для корректной работы графиков нужно установить нашу библиотеку с GitHub
!pip install git+https://github.com/ivanrychkov/toads

  Cloning https://github.com/ivanrychkov/toads to /private/var/folders/vt/3gzr9th52r1bvnt2w165_bpw0000gn/T/pip-req-build-fybutoye
  Created wheel for toads-ivanrychkov: filename=toads_ivanrychkov-0.0.8.2-py3-none-any.whl size=11367 sha256=79e63c046aed70f9e76242e0bc61c7c1582b330899107342112229a98fbcbfae
  Stored in directory: /private/var/folders/vt/3gzr9th52r1bvnt2w165_bpw0000gn/T/pip-ephem-wheel-cache-62hpkwk6/wheels/e2/3b/15/941c260ac8666ad497a66393d452203fcd98a6f55fbaa9ee37
Successfully built toads-ivanrychkov


In [2]:
import pandas as pd
from toads.image import Image
import seaborn as sns
import matplotlib.pyplot as plt

---

## Территориальное деление Москвы

- В качестве территориальной единицы мы взяли область вокруг почтового отделения.
- Чтобы объединить почты с районами и метро, Мы воспользовались Геокодером от Яндекса:
    - Скачали адреса всех почтовых отделений,
    - Получили координаты почт, районов и станций метро через API геокодера.
    - Объединили их с помощью кластеризации K-Means, подтянув координаты индексов к координатам районов и метро.
    - В результате у нас получился датасет из почтовых отделений, районов и станций метро.

In [3]:
pd.read_csv('geo_full.csv').sample(5)

,post_idx,latitude_post,longitude_post,district_idx,district,latitude_dsct,longitude_dsct,metro_idx,metro,latitude_mtr,longitude_mtr
232,121099,55.749435,37.582179,3,Арбат,55.751138,37.590003,173,Смоленская,55.748756,37.582161
349,117588,55.605241,37.530822,141,Ясенево,55.607514,37.534577,234,Ясенево,55.606182,37.533400
124,105122,55.804273,37.762210,63,Метрогородок,55.823616,37.755832,92,Локомотив,55.803130,37.746058
25,108807,55.521710,37.220957,86,Первомайское,55.519723,37.218334,158,Рассказовка,55.633968,37.334747
244,127422,55.814705,37.563808,117,Тимирязевский,55.825527,37.557071,190,Тимирязевская,55.818695,37.575271


---

## Подбор локации
- В базе данных **ФИАС** мы получили количество домов, привязанных к каждому почтовому индексу - благодаря этому не пришлось усреднять население на одну почту, а удалось посчитать примерную долю населения, привязанную к каждому отдельному индексу (`index_weight`).
- Это более точный подход, чем усреднение.

Таблица с населением на почтовый индекс выглядит вот так:

In [4]:
demo = pd.read_csv('index_people_data.csv').drop('sum', axis=1)
demo.person_exemption_group.fillna('', inplace=True)

In [5]:
demo.head()

,post_idx,index_weight,house_district,person_gender,Возрастной класс,person_exemption_group,index_sum
0,117036,0.11025,Академический,f,18-25,Многодетные,11
1,117036,0.11025,Академический,f,18-25,,429
2,117036,0.11025,Академический,f,25-45,Беременность,35
3,117036,0.11025,Академический,f,25-45,Инвалид,17
4,117036,0.11025,Академический,f,25-45,Малоимущие,0


Покажем на примере районов, как можно подбирать локацию по параметрам целевой аудитории.

In [6]:
def get_best_location(gender=None, age=None, exemptions=None, max_entries=None):
    global demo
    df = demo.copy()
    
    if gender is not None:
        if isinstance(gender, str):
            gender = [gender]
        df = df[df.person_gender.isin(gender)]
    if age is not None:
        if isinstance(age, str):
            age = [age]
        df = df[df['Возрастной класс'].isin(age)]
    if exemptions is not None:
        if isinstance(exemptions, str):
            exemptions = [exemptions]
        df = df[df.person_exemption_group.isin(exemptions)]
    
    group_args = [key for key, arg
                 in zip(
                     ('person_gender', 'Возрастной класс', 'person_exemption_group'),
                     (gender, age, exemptions))
                  if arg is not None]
    
    result = df.groupby(['house_district'] + group_args).agg({'index_sum': 'sum'})\
             .reset_index().drop(group_args, axis=1).groupby('house_district').sum()\
             .sort_values('index_sum', ascending=False).head(max_entries if max_entries else 5)
    
    if result.shape[0] > 0:
            with Image(st='Районы с релевантной аудиторией'):
                sns.barplot(x=result.index, y=result.index_sum / result.index_sum.max(), )
                plt.xticks(rotation=25)
                Image.format_axis(1)
                Image.labels('Район', 'Релевантность')
    else:
        print('Нет районов, соответствующих критериям!')

Ниже интерактивный вариант с диаграммами. Мы приводим не численность населения, а отмасштабированные от 0 до 1 значения, где 0 - минимум, а 1 - максимум. Это скроет числа и сделает вывод более интерпретируемым. Назовём это релевантностью.

In [7]:
import ipywidgets as widgets
from ipywidgets import interact

In [8]:
# Обозначим виджеты с выбором категорий
g_sel, age_sel, ex_sel = [widgets.SelectMultiple(options=list(v), value=tuple(v))
                          for v in [demo['person_gender'].unique(),
                                    demo['Возрастной класс'].unique(),
                                    demo['person_exemption_group'].unique()]]

In [9]:
print('\nВыберите интересующие категории:\n')
p = interact(get_best_location, age=age_sel, gender=g_sel, exemptions=ex_sel, max_entries=(1, 15))


Выберите интересующие категории:



interactive(children=(SelectMultiple(description='gender', index=(0, 1), options=('f', 'm'), value=('f', 'm'))…

##### Безусловно, если мы будем считать аудиторию не по районам, а по почтовым индексам, мы получим большую детализацию. И у нас эти данные есть.

---

## Скоринг
Аналогично тому, что мы показали выше, мы попробовали отмасштабировать информацию о поездках на метро и получили полный скоринг на примере районов. Эти данные можно легко применить к почтовым зонам, потому что у нас есть соответствующие ключи для связки.

In [10]:
metro_score = pd.read_csv('index_points.csv')

In [11]:
metro_score.head()

,metro,median_v_cnt,index_sum,arrival_cnt,inhabit_points,arrival_points,full_points
0,Пятницкое шоссе,122104.0,32944.0,89160.0,0.652956,0.008467,0.661423
1,Мякинино,171482.0,53468.0,118014.0,0.630472,0.011207,0.641679
2,Улица Скобелевская,137226.5,51053.0,86173.5,0.470928,0.008184,0.479112
3,Щёлковская,566744.0,2697.0,564047.0,0.016099,0.053565,0.069664
4,Хорошёво,69146.0,39645.0,29501.0,0.362937,0.002802,0.365738


---

## Вывод
- В связи с тем, что у нас ограничено время, мы постарались заложить основу для удобной работы с данными.
- Нашли способ объединить географические данные между собой через координаты. За единицу территории взяли почтовое отделение.
- Собрали датасет, в котором вывели приблизительную численность каждой аудитории для каждого почтового отделения.
- Предложили подбор локации, основанный на целевой аудитории и трафике метрополитена.
- Добавили возможность поэкспериментировать с подбором района по аудитории в интерактивном режиме.

---

## Что дальше?
У нас остались вопросы, на которые ещё предстоит ответить.

### Что, если все предприниматели бросятся в одно место?
- В ходе второго чек-ина возникла идея о том, что **предприниматели с одинаковыми товарами/услугами** могут заполонить район из-за того, что в нём присутствует целевая аудитория, и мы рекомендуем его всем подряд (самые населённые районы - Марьино и Южное Бутово - в зоне риска). **У нас есть идея, как этого не допустить**: наш сервис можно научить обращаться к API Яндекс.Карт или 2GIS при подборе локации, чтобы найти поблизости бизнесы с аналогичными услугами, на их основе сделать предположение о возможной конкуренции и включить это значение в скоринг. Например, *количество аналогичных бизнесов на одного потенциального пользователя*. **Высокая конкуренция - меньше рекомендуем, и наоборот**. Щепетильным моментом здесь является то, что бесплатно можно сделать лишь небольшое количество запросов к API картографических сервисов.

### Можно ли подобрать аудиторию автоматически по товару?
- Мы предполагаем, что возможной наводкой на аудиторию будет **семантика** в категориях товаров, например:
    - Игрушки - Дети
    - Лекарства - Пенсионеры
    - Бытовая техника - Молодая семья
    - Эконом-товары - Малоимущие
    
    С помощью правильной категоризации товаров и аудиторий и перевода их словесных названий в векторное представление мы можем попытаться **соотнести категории товаров/услуг с разными аудиториями** через кластеризацию по смысловым характеристикам слов.
- Как вариант, можно подбирать аудиторию, воспользовавшись готовыми данными, сформированными из поведения пользователей, или на основе опросов (например, во время подключения Wi-Fi в метро).

---